In [11]:
import math
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
# Import MNIST data
mnist = input_data.read_data_sets('./dataset/', one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting ./dataset/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting ./dataset/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./dataset/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./dataset/t10k-labels-idx1-ubyte.gz


In [2]:
save_file = './model/model.ckpt'

## Saving and Restoring a trained model

In [6]:
learning_rate = .01
n_input   = 784 # MNIST data input (img shape: 28*28)
n_classes = 10  # MNIST total classes (0-9 digits)

# Features and Labels
features = tf.placeholder(tf.float32, [None, n_input])
labels = tf.placeholder(tf.float32, [None, n_classes])

# Weights & bias
weights = tf.Variable(tf.random_normal([n_input, n_classes]))
bias = tf.Variable(tf.random_normal([n_classes]))

# Logits - xW + b
logits = tf.add(tf.matmul(features, weights), bias)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)\
    .minimize(cost)

In [9]:
# Calculate accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [14]:
batch_size = 128
n_epochs = 100

saver = tf.train.Saver()

In [15]:
# Launch the graph
with tf.Session() as sess:
    # Initialize all the Variables
    sess.run(tf.global_variables_initializer())
    
    # Training cycle
    for epoch in range(n_epochs):
        total_batch = math.ceil(mnist.train.num_examples / batch_size)
        
        # Loop over all batches
        for i in range(total_batch):
            batch_features, batch_labels = mnist.train.next_batch(batch_size)
            sess.run(
                optimizer,
                feed_dict={features:batch_features, labels:batch_labels})
        
        if epoch % 10 == 0:
            valid_accuracy = sess.run(accuracy,
                                     feed_dict={
                                         features: mnist.validation.images,
                                         labels: mnist.validation.labels
                                     })
            print(f'Epoch {epoch} - Validation Accuracy: {valid_accuracy}')
    
    # Save the model
    saver.save(sess, save_file)
    print('Trained Model Saved.')

Epoch 0 - Validation Accuracy: 0.2964000105857849
Epoch 10 - Validation Accuracy: 0.7391999959945679
Epoch 20 - Validation Accuracy: 0.7972000241279602
Epoch 30 - Validation Accuracy: 0.823199987411499
Epoch 40 - Validation Accuracy: 0.8396000266075134
Epoch 50 - Validation Accuracy: 0.850600004196167
Epoch 60 - Validation Accuracy: 0.8574000000953674
Epoch 70 - Validation Accuracy: 0.8632000088691711
Epoch 80 - Validation Accuracy: 0.8673999905586243
Epoch 90 - Validation Accuracy: 0.8726000189781189
Trained Model Saved.


In [16]:
with tf.Session() as sess:
    saver.restore(sess, save_file)
    
    test_accuracy = sess.run(
        accuracy, feed_dict={
            features: mnist.test.images,
            labels: mnist.test.labels
        })
    
    print(f'Test Accuracy: {test_accuracy}')

INFO:tensorflow:Restoring parameters from ./model/model.ckpt
Test Accuracy: 0.8776000142097473


# Using Dropout
During training, a good starting value for keep_prob is 0.5.

During testing, use a keep_prob value of 1.0 to keep all units and maximize the power of the model.

In [ ]:
keep_prob = tf.placeholder(tf.float32)

hidden_layer = tf.add(tf.matmul(features, weights[0]), biases[0])
hidden_layer = tf.nn.relu(hidden_layer)
hidden_layer = tf.nn.dropout(hidden_layer, keep_prob)

logits = tf.add(tf.matmul(hidden_layer, weights[1]), biases[1])